In [42]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torchinfo
from torchvision import transforms
from torch import nn
from going_modular.going_modular import data_setup,engine,helper_functions,model_builder,predictions,utils

In [43]:
print(torch.__version__)
print(torchvision.__version__)

2.2.1+cu121
0.17.1+cu121


In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [45]:
import os
from pathlib import Path
data_path = Path('../Datasets/')
image_path = data_path/'pizza_steak_sushi'
image_path.is_dir()

True

In [46]:
train_dir = image_path/'train'
test_dir = image_path/'test'

In [47]:
normalize= transforms.Normalize(mean=[0.485,0.456,0.406],
                                std=[0.229,0.224,0.225])
manual_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [48]:
train_dataloader,test_dataloader,class_names=data_setup.create_dataloaders(train_dir=train_dir,
                                                                           test_dir=test_dir,
                                                                           transform=manual_transform,
                                                                           batch_size=32)
train_dataloader,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x1b810ec30d0>,
 ['pizza', 'steak', 'sushi'])

In [49]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [50]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [51]:
train_dataloader,test_dataloader,class_names=data_setup.create_dataloaders(train_dir=train_dir,
                                                                           test_dir=test_dir,
                                                                           transform=auto_transforms,
                                                                           batch_size=32)
train_dataloader,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x1b7d0de2390>,
 ['pizza', 'steak', 'sushi'])

In [52]:
model = torchvision.models.efficientnet_b0(weights=weights)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [53]:
model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [54]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [55]:
from torchinfo import summary
summary(model = model,input_size=(1,3,224,224),col_names=['input_size','output_size','num_params','trainable'],col_width=20,row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 1000]            --                   True
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   True
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    864                  True
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    64                   True
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16, 112,

In [56]:
for param in model.features.parameters():
    param.requires_grad=False

In [57]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names))
).to(device)

In [58]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [59]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr = 0.001)

In [60]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
writer

In [65]:
from going_modular.going_modular.engine import train_step,test_step
from typing import Dict,List,Tuple
from tqdm.auto import tqdm


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        #Experiment tracking
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={'train_loss':train_loss,
                                            'test_loss':test_loss},
                           global_step = epoch)
        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={
                               'train_acc':train_acc,
                               'test_acc':test_acc
                           },
                           global_step = epoch)
        writer.add_graph(model=model,
                        input_to_model = torch.randn(32,3,224,224).to(device))
    writer.close()

    # Return the filled results at the end of the epochs
    return results

In [66]:
#Train model not using engine.py
results = train(model = model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=10,
                device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5977 | train_acc: 0.7812 | test_loss: 0.6352 | test_acc: 0.7945


 10%|█         | 1/10 [00:22<03:20, 22.22s/it]

Epoch: 2 | train_loss: 0.5345 | train_acc: 0.7773 | test_loss: 0.5639 | test_acc: 0.8258


 20%|██        | 2/10 [00:44<02:59, 22.50s/it]

...

 80%|████████  | 8/10 [03:16<00:50, 25.33s/it]

Epoch: 9 | train_loss: 0.3820 | train_acc: 0.8359 | test_loss: 0.4435 | test_acc: 0.8153


 90%|█████████ | 9/10 [03:41<00:25, 25.25s/it]

Epoch: 10 | train_loss: 0.4351 | train_acc: 0.8320 | test_loss: 0.4468 | test_acc: 0.8153


100%|██████████| 10/10 [04:05<00:00, 24.52s/it]


In [67]:
#Viewing the experiments 
%reload_ext tensorboard
%tensorboard --logdir runs/

In [64]:
results

{'train_loss': [1.0786151364445686,
  0.8544604331254959,
  0.7336534932255745,
  0.7233732715249062,
  0.5828549154102802],
 'train_acc': [0.39453125, 0.6953125, 0.8125, 0.69921875, 0.91796875],
 'test_loss': [0.8163254658381144,
  0.7031100591023763,
  0.6093792915344238,
  0.5786730547746023,
  0.6003653804461161],
 'test_acc': [0.7840909090909092,
  0.8456439393939394,
  0.875,
  0.8655303030303031,
  0.8456439393939394]}

In [69]:
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name:str,
                  model_name:str,
                  extra:str = None):
    from datetime import datetime
    import os
    timestamp = datetime.now().strftime("%Y-%m-%d")
    if extra:
        log_dir = os.path.join("runs",timestamp,experiment_name,model_name,extra)
    else:
        log_dir = os.path.join("runs",timestamp,experiment_name,model_name)
    print(f"Created summarywriter saved to {log_dir}")
    return SummaryWriter(log_dir=log_dir)

In [70]:
from going_modular.going_modular.engine import train_step,test_step
from typing import Dict,List,Tuple
from tqdm.auto import tqdm


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer:torch.utils.tensorboard.SummaryWriter) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        if writer:
        #Experiment tracking
            writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={'train_loss':train_loss,
                                            'test_loss':test_loss},
                           global_step = epoch)
            writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={
                               'train_acc':train_acc,
                               'test_acc':test_acc
                           },
                           global_step = epoch)
            writer.add_graph(model=model,
                        input_to_model = torch.randn(32,3,224,224).to(device))
        else:
            pass
    if writer:
        writer.close()
    else:
        pass
    # Return the filled results at the end of the epochs
    return results

In [71]:
results = train(model = model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=3,
                device=device,
                writer = create_writer('food','vgg'))

Created summarywriter saved to runs\2024-03-11\food\vgg


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.3265 | train_acc: 0.9805 | test_loss: 0.4133 | test_acc: 0.8561


 33%|███▎      | 1/3 [00:23<00:47, 23.98s/it]

Epoch: 2 | train_loss: 0.4269 | train_acc: 0.8242 | test_loss: 0.3847 | test_acc: 0.8456


 67%|██████▋   | 2/3 [00:48<00:24, 24.04s/it]

Epoch: 3 | train_loss: 0.3439 | train_acc: 0.9688 | test_loss: 0.3889 | test_acc: 0.8665


100%|██████████| 3/3 [01:14<00:00, 24.76s/it]
